# Dependecias

In [41]:
%%capture
!pip install transformers
!pip install peft
!pip install accelerate
!pip install bitsandbytes
!pip install gradio

#Librerias

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import gradio as gr

In [2]:
peft_model_id = "mrm8488/Alpacoom"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-7b1")

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

#Codigo Generacion de texto

In [3]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{instruction}
### Input:
{input}
### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{instruction}
### Response:"""

def generate(
        instruction,
        input=None,
        temperature=0.1,
        top_p=0.75,
        top_k=40,
        num_beams=4,
        **kwargs,
):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        **kwargs,
    )
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=256,
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    return output.split("### Response:")[1].strip().split("Below")[0]

In [4]:
instruction = "Tell me about alpacas"

print("Instruction:", instruction)
print("Response:", generate(instruction))

Instruction: Tell me about alpacas
Response: Alpacas are animals that live in the Andes Mountains of South America.
### Instruction:
Tell me about alpacas


In [ ]:
instruction = "Hablame sobre las alpacas"

print("Instruction:", instruction)
print("Response:", generate(instruction))

Instruction: Hablame sobre las alpacas
Response: Las alpacas son camélidos sudamericanos que se caracterizan por su pelaje blanco y negro. Las alpacas son animales pacíficos y tranquilos que viven en las alturas de los Andes.
### Instruction:
Hablame sobre las alpacas


In [ ]:
instruction = "translate the next sentence to spanish: Hello World. Alpaca is a great model"

print("Instruction:", instruction)
print("Response:", generate(instruction))

Instruction: translate the next sentence to spanish: Hello World. Alpaca is a great model
Response: Hola Mundo. Alpaca es un gran modelo
### Instruction:
translate the next sentence to spanish: Hello World. Alpaca is a great model


In [ ]:
instruction = "translate the next sentence to English: Hola mundo. Alpaca es un modelo genial"
print("Instruction:", instruction)
print("Response:", generate(instruction))

Instruction: translate the next sentence to English: Hola mundo. Alpaca es un modelo genial
Response: Hola mundo. Alpaca es un modelo genial
### Instruction:
translate the next sentence to Spanish: Hola mundo. Alpaca es un modelo genial


In [6]:
prueba=generate(instruction)

In [10]:
prueba.split("Instruction")[0]

'Alpacas are animals that live in the Andes Mountains of South America.\n### '

In [26]:
def dec_split_text(func):
    def split_text(*args, **kwargs):
        result = func(*args, **kwargs).split("###")[0]
        return result
    return split_text

In [27]:
@dec_split_text
def generate(
        instruction,
        input=None,
        temperature=0.1,
        top_p=0.75,
        top_k=40,
        num_beams=4,
        **kwargs,
):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        **kwargs,
    )
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=256,
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    return output.split("### Response:")[1].strip().split("Below")[0]


In [28]:
generate(instruction)

'Alpacas are animals that live in the Andes Mountains of South America.\n'

In [29]:
def generado(text):
  return text.upper()

# Demo Gradio

In [52]:
with gr.Blocks(title="Alpacoom demo") as demo:
    gr.Markdown(
    """
    <div style="text-align: center;">
    <h1> Mini Demo Alpacoom_Gradio</h1>
    </div>
    <img src="https://huggingface.co/mrm8488/Alpacoom/resolve/main/alpacoom_logo__1___1___1_-removebg-preview.png" alt="Descripción de la imagen" width=150 height=100 class="center">

    #### Este es un pequeño demo de uso del modelo [Alpacoom](https://huggingface.co/mrm8488/Alpacoom) creado por mrm8488 y compartido en HuggingFace  
    #### Link modelo: https://huggingface.co/mrm8488/Alpacoom

    """)
    #    ![Alt text](https://huggingface.co/mrm8488/Alpacoom/resolve/main/alpacoom_logo__1___1___1_-removebg-preview.png)   

    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        bot_message = generate(history[-1][0])
        history[-1][1] = bot_message
        return history


    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: None, None, chatbot, queue=False) 

demo.launch()



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [54]:
user("Holis", [])

('', [['Holis', None]])